In [194]:
import cv2
import pytesseract
import re
import glob
import os
import numpy as np

In [195]:
#install tesseract: https://github.com/UB-Mannheim/tesseract/wiki
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

In [196]:
np.set_printoptions(precision=2)

In [197]:
path = r'data'
odds_sequences = glob.glob(os.path.join(path, "*.jpg"))

In [198]:
odds_sequences

['data\\matrix-1.JPG', 'data\\matrix-2.JPG', 'data\\matrix-3.JPG']

In [199]:
def img_to_float_array(filepath): 
    # Read the image
    img = cv2.imread(filepath)

    # Apply some pre-processing (optional)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    #gray = cv2.medianBlur(gray, 3)

    # Use Tesseract OCR to extract text from the image
    text = pytesseract.image_to_string(gray, config='--psm 6')

    # Extract numbers from the text
    sequence = [int(s) for s in text.split() if s.isdigit()]

    # Use regular expression to split the string by non-digit characters or dot
    sequence = re.split(r'[^0-9.]', text)

    # Filter out empty strings and convert the list of strings to floats
    #numbers = [float(num) for num in numbers if num]
    sequence = [s for s in sequence if s]

    # Clean values
    sequence = [s for s in sequence if s not in ['0','.']]

    # If a string value does not contain a dot, insert a dot after the second digit from the right
    sequence = [s[: -2] + '.' + s[-2:] if '.' not in s else s for s in sequence]

    # Reverse the order of the numbers (bottom right to top left)
    sequence.reverse()
    
    #print(sequence)

    # transform to float and clean further
    sequence = [float(s) for s in sequence]
    sequence = [f for f in sequence if f >= 1]
    
    return sequence

In [200]:
def calculate_session_win(odds_array, left_bet, left_odd, right_bet, right_odd, recover_steps, start_after_first_defeat=True):
    started = False
    mode = 'bet'
    profit = 0
    steps = recover_steps
    win = 0
    max_win = 0
    max_loss = 0
    
    for odd in odds_array:
        balance = win
        if not started:
            if start_after_first_defeat:
                if odd >= left_odd:
                    continue
                else:
                    started=True
                    continue
            else:
                started=True
        if mode == 'bet':
            if odd >= left_odd:
                win += (left_bet*left_odd) - left_bet
            else:
                win -= left_bet
                mode = 'recover'
            
            if odd >= right_odd:
                win += (right_bet*right_odd) - right_bet
            else:
                win -= right_bet
        elif mode == 'recover':
            if odd >= left_odd:
                win += (left_bet*left_odd) - left_bet
                steps -= 1
            else:
                win -= left_bet
                steps = steps + recover_steps
            
            if steps == 0:
                steps = recover_steps
                mode = 'bet'
        if win > max_win:
            max_win = win
        if win < max_loss:
            max_loss = win    
        #print('Balance: ' + str(balance) + ', Odd: '+ str(odd) + ', Mode: ' + mode + ', Steps: ' + str(steps) + ', Win: ' + str(win))     
    return win, max_win, max_loss      

In [201]:
# list of bet configuration to be evaluated
configs = []
for left_odd, recover_steps in [(1.10, 10), (1.20, 5), (1.25, 4), (1.33, 3)]:
    for right_odd in [5, 10, 25, 50, 70, 100, 200]:
        configs.append({'left_bet':10, 'left_odd':left_odd, 'right_bet':1, 'right_odd':right_odd, 'recover_steps':recover_steps, 'start_after_first_defeat':True})

In [202]:
for c_idx, c in enumerate(configs):
    for seq_idx, seq in enumerate(odds_sequences):
        win, max_win, max_loss = calculate_session_win(img_to_float_array(seq), 1, c['left_odd'], 1, c['right_odd'], c['recover_steps'], c['start_after_first_defeat'])
        if win>=0:
            print('Config: ' + str(c_idx) + ', Seq: '+ str(seq_idx) + ', Win: ' + str(win) + ', Max. win: ' + str(max_win) + ', Max. loss: ' + str(max_loss))    

Config: 0, Seq: 2, Win: 4.899999999999986, Max. win: 8.2, Max. loss: 0
Config: 1, Seq: 2, Win: 14.899999999999993, Max. win: 18.2, Max. loss: 0
Config: 7, Seq: 2, Win: 4.0000000000000036, Max. win: 10.2, Max. loss: -2.4000000000000004
Config: 8, Seq: 2, Win: 8.999999999999972, Max. win: 15.2, Max. loss: -2.4000000000000004
Config: 9, Seq: 2, Win: 13.999999999999972, Max. win: 21.8, Max. loss: -2.4000000000000004
Config: 10, Seq: 2, Win: 39.000000000000114, Max. win: 46.8, Max. loss: -2.4000000000000004
Config: 11, Seq: 2, Win: 59.000000000000114, Max. win: 66.8, Max. loss: -2.4000000000000004
Config: 12, Seq: 2, Win: 89.00000000000011, Max. win: 96.8, Max. loss: -2.4000000000000004
Config: 13, Seq: 2, Win: 188.99999999999955, Max. win: 196.8, Max. loss: -2.4000000000000004
Config: 14, Seq: 2, Win: 3.75, Max. win: 10.5, Max. loss: -2.25
Config: 15, Seq: 2, Win: 8.75, Max. win: 15.5, Max. loss: -2.25
Config: 16, Seq: 2, Win: 13.75, Max. win: 22.0, Max. loss: -2.25
Config: 17, Seq: 2, Win